# Project 1 - part A

Dataset with the 30 Stocks from S&P500 Index from 2014 to 2024

In [20]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../project1-partA/project1_data.csv")
df_SP500 = pd.read_csv("../project1-partA/sp500_px_open_last.csv")
df = df.dropna() # VRTX has null PX_LAST in 2015-05-12	
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75479 entries, 0 to 75479
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          75479 non-null  object 
 1   PX_LAST       75479 non-null  float64
 2   CUR_MKT_CAP   75479 non-null  float64
 3   PX_VOLUME     75479 non-null  float64
 4   Ticker        75479 non-null  object 
 5   Market_Value  75479 non-null  float64
dtypes: float64(4), object(2)
memory usage: 4.0+ MB


## Daily Return

In [21]:
tickers = df["Ticker"].unique()
all_tickers_df = []

for tik in tickers:
    mask = df["Ticker"]==tik
    last_day_value = df.loc[mask,'PX_LAST'].shift(1)
    today_value = df.loc[mask,'PX_LAST']
    df.loc[mask, 'Daily_Return'] = (last_day_value - today_value) / last_day_value

df.loc[df['Daily_Return'].isna(),"Daily_Return"] = 0
df

,Date,PX_LAST,CUR_MKT_CAP,PX_VOLUME,Ticker,Market_Value,Daily_Return
0,2014-01-02,76.27,134027.7199,5212460.0,DIS,1.022229e+07,0.000000
1,2014-01-03,76.11,133746.5552,4061042.0,DIS,1.017945e+07,0.002098
2,2014-01-06,75.82,133236.9441,6816693.0,DIS,1.010203e+07,0.003810
3,2014-01-07,76.34,134150.7295,4511157.0,DIS,1.024107e+07,-0.006858
4,2014-01-08,75.22,132182.5763,10914858.0,DIS,9.942773e+06,0.014671
...,...,...,...,...,...,...,...
75475,2023-12-22,80.43,15764.7189,835062.0,TXT,1.267956e+06,-0.004496
75476,2023-12-26,80.73,15823.5205,678910.0,TXT,1.277433e+06,-0.003730
75477,2023-12-27,80.71,15819.6004,899628.0,TXT,1.276800e+06,0.000248
75478,2023-12-28,80.75,15827.4407,603963.0,TXT,1.278066e+06,-0.000496


## Value-Weighted and Equally-Weighted Portfolio

In [22]:
n_tickers = len(df["Ticker"].unique())

df['Date'] = pd.to_datetime(df['Date'])

# Group by year and month
grouped = df.groupby([df['Date'].dt.year, df['Date'].dt.month])

# This is a list of dataframes grouped by the month and year of the Date
dfs_by_month_year = [group.reset_index(drop=True) for _, group in grouped]

# Setting the first portfolio weight
equal_weight = 1 / n_tickers
dfs_by_month_year[0]["Equal_Weight"] = equal_weight

# This is the value that we start with (100%)
cumulative_value = 1

for i in range(len(dfs_by_month_year)-1):
    # Get the dataset from this specific year and month
    a = dfs_by_month_year[i]
    
    # Calculate the portfolio daily return
    a["Equal_weight*Daily_Return"] = a["Equal_Weight"]*a["Daily_Return"]
    EWP_daily_returns = a.groupby("Date")["Equal_weight*Daily_Return"].sum().rename("EWP_daily_return")
    dfs_by_month_year[i] = a.drop(columns=["Equal_weight*Daily_Return"])
    # Update the cumulative return after this month
    for EWP_daily_return in EWP_daily_returns:
        cumulative_value*=(1+EWP_daily_return)
        
    # Use the cumulative value at the end of the month and divide it by the number of tickers
    next_month_Equal_weight = cumulative_value / n_tickers
    
    # Updating the value at for the next month
    b = dfs_by_month_year[i+1]
        
    b["Equal_Weight"] = next_month_Equal_weight

In [23]:
# Connecting all the dataframes
new_df = pd.concat(dfs_by_month_year)
new_df

,Date,PX_LAST,CUR_MKT_CAP,PX_VOLUME,Ticker,Market_Value,Daily_Return,Equal_Weight
0,2014-01-02,76.27,134027.7199,5212460.0,DIS,1.022229e+07,0.000000,0.033333
1,2014-01-03,76.11,133746.5552,4061042.0,DIS,1.017945e+07,0.002098,0.033333
2,2014-01-06,75.82,133236.9441,6816693.0,DIS,1.010203e+07,0.003810,0.033333
3,2014-01-07,76.34,134150.7295,4511157.0,DIS,1.024107e+07,-0.006858,0.033333
4,2014-01-08,75.22,132182.5763,10914858.0,DIS,9.942773e+06,0.014671,0.033333
...,...,...,...,...,...,...,...,...
595,2023-12-22,80.43,15764.7189,835062.0,TXT,1.267956e+06,-0.004496,0.011832
596,2023-12-26,80.73,15823.5205,678910.0,TXT,1.277433e+06,-0.003730,0.011832
597,2023-12-27,80.71,15819.6004,899628.0,TXT,1.276800e+06,0.000248,0.011832
598,2023-12-28,80.75,15827.4407,603963.0,TXT,1.278066e+06,-0.000496,0.011832
